In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import gzip
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import time

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
print('ok')

ok


In [2]:
cnt=0
for l in readGz("data/train.json.gz"):
    print(l)
    cnt+=1
    if cnt>=5:
        break

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}
{'rating': 5.0, 'businessID': 'B410583770', 'reviewText': "Quaint, stylish atmosphere in a small place right off South Congress. The food and service were great. It's pricey, but the happy hour deals are considerable. Great for a date!", 'userID': 'U637588426', 'reviewTime': 'Sep 21, 2011', 'categories': ['Italian Restaurant', 'European Restaurant'], 'reviewHash': 'R954562590', 'unixReviewTime': 1316616175}
{'rating': 4.0, 'businessID': 'B423621081', 'reviewText': "I'm a vegetarian, but every so often I want a hotdog with lots of toppings.  And a tall can of beer.  F

In [3]:
# new_cata={}
# cata_file=open('data/cata/new_cata.txt')
# for line in cata_file:
#     l=line.strip().split('\t')
#     new_cata[l[0]]=l[1]

# def trans_cata(ca):
#     if ca in new_cata:
#         return new_cata[ca]
#     else:
#         return ca

In [4]:
users=[]
bussiness=[]
catas=[]
for l in readGz("data/train.json.gz"):
    u=l['userID']
    b=l['businessID']
    c=l['categories']
    users.append(u)
    bussiness.append(b)
    catas+=c
    
user_set=set(users)
buss_set=set(bussiness)
cata_set=set(catas)

Nu=len(user_set)  # 18793
Nb=len(buss_set)  # 21321
Nc=len(cata_set)  # 1377

user_index={}
buss_index={}
cata_index={}

for (i,u) in enumerate(user_set):
    user_index[u]=i
for (i,b) in enumerate(buss_set):
    buss_index[b]=i
for (i,c) in enumerate(cata_set):
    cata_index[c]=i
print(Nu,Nb,Nc)
print('ok')

18793 21321 1377
ok


In [5]:
num_train=199000

user_cata_train=np.zeros((Nu,Nc))
buss_cata_train=np.zeros((Nb,Nc))
user_business_train={}

user_business_val={}

user_cata_all=np.zeros((Nu,Nc))
buss_cata_all=np.zeros((Nb,Nc))
user_business_all={}

ui_val=[]
bi_val=[]
y_val=[]

i=0
for l in readGz("data/train.json.gz"):
    u=user_index[l['userID']]
    b=buss_index[l['businessID']]
    if i<num_train:
        if u in user_business_train:
            user_business_train[u].append(b)
        else:
            user_business_train[u]=[b]
        for c in l['categories']:
            c_i=cata_index[c]
#             user_cata_train[u][c_i]+=1
            user_cata_train[u][c_i]=1
            buss_cata_train[b][c_i]=1
    else:
        ui_val.append(u)
        bi_val.append(b)
        y_val.append(1)
        if u in user_business_val:
            user_business_val[u].append(b)
        else:
            user_business_val[u]=[b]
    if u in user_business_all:
        user_business_all[u].append(b)
    else:
        user_business_all[u]=[b]
    for c in l['categories']:
        c_i=cata_index[c]
#         user_cata_all[u][c_i]+=1
        user_cata_all[u][c_i]=1
        buss_cata_all[b][c_i]=1
    i+=1
print('ok')

ok


In [6]:
mode='train'

In [7]:
def sigmoid(X):
    return 1.0/(1.0+np.exp(-X))

if mode=='train':
    user_total={}
    buss_total={}
    y=np.zeros((Nu,Nb),dtype=np.bool_)
    val_mask=np.ones((Nu,Nb),dtype=np.bool_)
    for (u,v) in user_business_train.items():
        for b in v:
            user_total[u]=1 if u not in user_total else user_total[u]+1
            buss_total[b]=1 if b not in buss_total else buss_total[b]+1
            y[u,b]=True
    for (u,v) in user_business_val.items():
        for b in v:
            val_mask[u,b]=False

    K=10
    weight_scale=1
    max_iter=200
    print_every=10
    lamb=1e-2
    # gamma_u=np.random.randn(Nu,K).astype(np.float32)*weight_scale
    # gamma_b=np.random.randn(Nb,K).astype(np.float32)*weight_scale
#     gamma_u=np.random.randn(Nu,K)*weight_scale
#     gamma_b=np.random.randn(Nb,K)*weight_scale
    gamma_u=np.random.rand(Nu,K)
    gamma_b=np.random.rand(Nb,K)
    # learing_rate=1000/(Nu*Nb)
    learing_rate=1e-5

    # use all yij to update
    # for it in range(max_iter):
    #     for i in range(Nu):
    #         sig=sigmoid(np.sum(gamma_u[i,:]*gamma_b,axis=1))
    #         d_gamma_u_i=((y[i,:]-sig)*val_mask[i,:]).reshape((1,Nb)).dot(gamma_b).reshape(K,)/user_total[u]
    #         gamma_u[i,:]+=learing_rate*d_gamma_u_i
    #     loss=0.0
    #     for j in range(Nb):
    #         sig=sigmoid(np.sum(gamma_b[j,:]*gamma_u,axis=1))
    #         d_gamma_b_j=((y[:,j]-sig)*val_mask[:,j]).reshape((1,Nu)).dot(gamma_u).reshape(K,)/buss_total[b]
    #         gamma_b[j,:]+=learing_rate*d_gamma_b_j
    #         loss+=np.sum((y[:,j]*np.log(sig)+(1.0-y[:,j])*np.log(1.0-sig))*val_mask[:,j])/(Nu*Nb)
    # #     if it==0 or (it+1)%print_every==0:
    #     print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),'iter %d, loss=%.10f' %(it+1,loss))

    for it in range(max_iter):
        gamma_u=y.dot(gamma_b.dot(np.linalg.inv(gamma_b.T.dot(gamma_b))))
        gamma_b=y.T.dot(gamma_u.dot(np.linalg.inv(gamma_u.T.dot(gamma_u))))

        loss=np.sum(np.square(gamma_u.dot(gamma_b.T)-y))
    #     for j in range(Nb):
    #         sig=sigmoid(np.sum(gamma_b[j,:]*gamma_u,axis=1))
    #         d_gamma_b_j=((y[:,j]-sig)*val_mask[:,j]).reshape((1,Nu)).dot(gamma_u).reshape(K,)/buss_total[b]
    #         gamma_b[j,:]+=learing_rate*d_gamma_b_j
    #         loss+=np.sum((y[:,j]*np.log(sig)+(1.0-y[:,j])*np.log(1.0-sig))*val_mask[:,j])/(Nu*Nb)
        if it==0 or (it+1)%print_every==0:
            print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),'iter %d, loss=%.10f' %(it+1,loss))
    np.save('data/gamma_u',gamma_u)
    np.save('data/gamma_b',gamma_b)

else:
    gamma_u=np.load('data/gamma_u.npy')
    gamma_b=np.load('data/gamma_b.npy')

2017-11-08 10:07:20 iter 1, loss=197990.9940943087
2017-11-08 10:10:13 iter 10, loss=192316.4972478374
2017-11-08 10:13:22 iter 20, loss=192290.8768477430
2017-11-08 10:16:32 iter 30, loss=192288.8249517865
2017-11-08 10:19:43 iter 40, loss=192288.6106569871
2017-11-08 10:24:31 iter 50, loss=192288.5673629546
2017-11-08 10:27:49 iter 60, loss=192288.5505729645
2017-11-08 10:31:07 iter 70, loss=192288.5423840758
2017-11-08 10:34:25 iter 80, loss=192288.5382074020
2017-11-08 10:37:44 iter 90, loss=192288.5360617352
2017-11-08 10:40:54 iter 100, loss=192288.5349583499
2017-11-08 10:44:04 iter 110, loss=192288.5343909038
2017-11-08 10:47:13 iter 120, loss=192288.5340990912
2017-11-08 10:50:21 iter 130, loss=192288.5339490309
2017-11-08 10:53:30 iter 140, loss=192288.5338718647
2017-11-08 10:56:38 iter 150, loss=192288.5338321806
2017-11-08 10:59:47 iter 160, loss=192288.5338117763
2017-11-08 11:02:55 iter 170, loss=192288.5338012847
2017-11-08 11:06:02 iter 180, loss=192288.5337958859
2017

In [8]:
print(gamma_u)
print(gamma_b)
def predict_y(u,b):
    print(sigmoid(np.inner(gamma_u[u,:],gamma_b[b,:])))
    return sigmoid(np.inner(gamma_u[u,:],gamma_b[b,:]))>0.5

print(predict_y(user_index['U637588426'],buss_index['B408037852']))

# for i in range(10):
#     for j in range(10):
#         u=user_index[users[i]]
#         b=buss_index[bussiness[j]]
#         print(i,j,sigmoid(np.inner(gamma_u[u,:],gamma_b[b,:])))



[[ -1.47099762e-04   8.47598195e-05   2.01379352e-04 ...,  -1.76658291e-04
    1.53225584e-04  -5.37970891e-05]
 [ -1.12808520e-04   6.00433892e-05   1.53966566e-04 ...,  -1.35224977e-04
    1.17357444e-04  -4.01190413e-05]
 [ -8.22681742e-05   4.78475938e-05   1.11777756e-04 ...,  -9.89825276e-05
    7.99630058e-05  -2.61697541e-05]
 ..., 
 [ -1.41872960e-04  -6.83829488e-05   2.15780532e-05 ...,  -5.15201919e-05
   -2.04160694e-04   1.86661866e-04]
 [  1.14207389e-04   1.62399326e-03  -1.00746663e-03 ...,   5.18100975e-04
    6.48948132e-04  -1.24547716e-03]
 [ -1.76501366e-96   1.43057856e-96   3.45984593e-97 ...,   8.80831713e-97
   -1.16332741e-97  -5.91271982e-97]]
[[ 0.45711102  7.17239794  3.79442217 ...,  3.0279491   2.4248393
   2.40248014]
 [ 0.90756516  0.62084677  0.13644819 ...,  1.44322484  0.14341982
   1.09262061]
 [ 0.01252168  0.05094495  0.00888877 ...,  0.16936262  0.18844412
   0.06132108]
 ..., 
 [ 0.22403181  0.27541513  0.1181501  ...,  0.11777271 -0.4143004
  

In [9]:
# X_user=PCA(n_components=100).fit_transform(user_cata_train)
# user_labels=100
# y_user = KMeans(n_clusters=user_labels,max_iter=1000,n_jobs=4).fit_predict(X_user)
# print('ok')
# X_buss=PCA(n_components=100).fit_transform(buss_cata_train)
# buss_labels=100
# y_buss = KMeans(n_clusters=buss_labels,max_iter=1000,n_jobs=4).fit_predict(X_buss)
# print('ok')
# label_catas=[]
# for i in range(user_labels):
#     index=np.where(y_user==i)[0]
#     label_catas.append(np.sum(user_cata[index,:],axis=0))
# print('ok')
# UB=np.zeros((user_labels,buss_labels))
# for (u,buss_list) in user_business_train.items():
#     u_label=y_user[u]
#     for b in buss_list:
#         b_label=y_buss[b]
#         UB[u_label,b_label]+=1
# sort_indexs=[]
# for i in range(user_labels):
#     temp=UB[i,:]
#     sii=list(np.argsort(temp)[::-1])
#     sort_indexs.append(sii)
# print('ok')

In [10]:
num_val=(len(users)-num_train)*2
ui_val_first=np.array(ui_val)
bi_val_first=np.array(bi_val)
ui_val_second=np.random.randint(Nu,size=(num_val//2,))
bi_val_second=np.random.randint(Nb,size=(num_val//2,))
for i in range(ui_val_second.shape[0]):
    y_val.append(bi_val_second[i] in user_business_all[ui_val_second[i]])
ui_val=np.concatenate((ui_val_first,ui_val_second))
bi_val=np.concatenate((bi_val_first,bi_val_second))
y_val=np.array(y_val)
print(ui_val.shape)
print(bi_val.shape)
print(np.sum(y_val))

(2,)
(2,)
1


In [11]:
true_pos=0
true_neg=0
false_pos=0
false_neg=0

for i in range(num_val):
    ui=ui_val[i]
    bi=bi_val[i]
    real=y_val[i]
    
    if np.sum(user_cata_train[ui])==0 or np.sum(buss_cata_train[bi])==0:
        predict=random.randint(0,1)
    elif bi in user_business_train[ui]:
        predict=1
    else:
#         predict=np.sum(user_cata_train[ui]*buss_cata_train[bi])>0
        predict=np.inner(gamma_u[ui,:],gamma_b[bi,:])>0.00015
        
    if predict==1 and real==1:
        true_pos+=1
    elif predict==1 and real==0:
        false_pos+=1
    elif predict==0 and real==1:
        false_neg+=1
    elif predict==0 and real==0:
        true_neg+=1
print('accuracy =',(true_pos+true_neg)/num_val)
print('true pos =',true_pos)
print('false pos =',false_pos)
print('false neg =',false_neg)
print('true neg=',true_neg)
pp=true_pos/(true_pos+false_pos)
rr=true_pos/(true_pos+false_neg)
print('F1 =',(2.0*pp*rr)/(pp+rr))

accuracy = 0.5
true pos = 0
false pos = 0
false neg = 1
true neg= 1


ZeroDivisionError: division by zero

In [ ]:
# label_cata=label_catas[0]
# print(list(label_cata))
# num_tot_cata=np.sum(label_cata)
# sort_index=np.argsort(label_cata)[::-1]
# cata_cnt=0
# id=0
# while cata_cnt<num_tot_cata*0.8:
#     cata_cnt+=label_cata[sort_index[id]]
#     id+=1
# print(list(np.sort(label_cata)[::-1]))
# print(id)

In [12]:
test_file=open('data/pairs_Visit.txt')
fo=open('data/pairs_Visit_out2.txt','w')
fo.write('userID-businessID,prediction\n')
c1=0
c2=0
c3=0
for line in test_file:
    if line == 'userID-businessID,prediction\n':
        continue
    [u,b]=line.strip().split('-')
    if u not in user_index or b not in buss_index:
        predict=random.randint(0,1)
        fo.write('%s-%s,%d\n' %(u,b,predict))
        c1+=1
    elif buss_index[b] in user_business_all[user_index[u]]:
        fo.write('%s-%s,1\n' %(u,b))
        c2+=1
    else:
        ui=user_index[u]
        bi=buss_index[b]
        
#         predict=np.sum(user_cata_all[ui]*buss_cata_all[bi])>0

        predict=np.inner(gamma_u[ui,:],gamma_b[bi,:])>0.0001
                
        fo.write('%s-%s,%d\n' %(u,b,predict))
        c3+=1
fo.close()
print(c1,c2,c3)

63 11 39926


In [ ]:
# a=np.array([2,5,3,6,7,1,0,0,0,0])
# tt=np.argsort(a)
# print(tt[::-1])